**Disclaimer**: functions and methods used in this notebook are covered in the next lessons. Here it is only to generate the tables to show different versions of storing data. If you want to study these examples, come back after the next few notebooks.

In [ ]:
import pandas as pd
print(pd.__version__)

import numpy as np

import os

# Organizing data

Topics we will discuss in this lesson are closely related to database management. The concept of *tidy data* was introduced by Hadley Wickham (Chief scientist in **R** project) and was inspired by databases. It turns out that scientists and statisticians can benifit very much from the same concepts. In particular, structuring your data in a tidy way will facilitate any type of analysis you want to do.

The core ideas in this lesson are taken from Hadley Wickham's seminal paper "Tidy data" [1], which IMO every person who works with complex datasets should read. Some examples in this notebook are from that paper, while others I made up myself.

[1] Wickham, H. (2014). Tidy data. Journal of Statistical Software, 59(10), 1-23.

# Tidying: structuring datasets to facilitate analysis

The principles of tidy data provide a standard way to organize data values within a dataset. Current tools often require translation: you have to spend time munging the output from one tool so you can input it into another. Tidy datasets and tools for them work hand in hand to make data analysis easier, allowing you to focus on the interesting domain problem, not on the uninteresting logistics of data.

Let's start with an example.

Consider the following 2 ways of presenting the same toy data. Think about whether there is any difference how we organize it.

In [ ]:
untidy = pd.DataFrame({'treatment_a':[np.nan, 16, 3],'treatment_b':[2,11,1]}, 
                      index=['John Smith', 'Jane Doe','Mary Johnson'])
untidy

In [ ]:
untidy.T

A dataset is a collection of values, usually either numbers (if quantitative) or strings (if
qualitative). Values are organized in two ways. Every value belongs to a variable and an
observation. **A variable contains all values that measure the same underlying attribute** (like
height, temperature, duration) across units. **An observation contains all values measured on
the same unit** (like a person, or a day, or a race) across attributes.

Let's restructure the dataset in the following (*tidy*, as we will learn later) way:

In [ ]:
untidy.index.name = 'person'
untidy.columns.name = 'treatment'
tidy = pd.melt(untidy.reset_index(),id_vars=['person'],value_name='result')
tidy['treatment'].replace({'treatment_a':'a','treatment_b':'b'}, inplace=True)
tidy

This makes it clear that the
dataset contains 18 values representing three variables and six observations. The variables
are:
1. person, with three possible values (John Smith, Mary Johnson, and Jane Doe).
2. treatment, with two possible values (a and b).
3. result, with five or six values depending on how you think of the missing value (—,
16, 3, 2, 11, 1).


# Tidy data
Tidy data is a standard way of mapping the meaning of a dataset to its structure. A dataset is
messy or tidy depending on how rows, columns and tables are matched up with observations,
variables and types. Core principles of tidy data are simple:
1. Each **variable** forms a **column**
2. Each **observation** forms a **row**
3. Each **type of observational unit** forms a **table**

In [ ]:
tidy

In [ ]:
untidy

In [ ]:
income_untidy = pd.read_csv(os.path.join('data','pew.csv'))
income_untidy

In [ ]:
income_untidy.shape

Here there is a `religion` column, which forms a separate variable, but all other columns actually contain the same variable -- `count`. `income` forms another ("groupping") variable:

In [ ]:
income_tidy = pd.melt(income_untidy,id_vars=['religion'],var_name='income',value_name='count')
income_tidy.head(25)

In [ ]:
income_tidy.shape

Some things I can do with untidy as well as with tidy, but what if I wanted to change to proportion for each religion to have certain income, but keep the count as well (because it tells me how precise is the measurement and is need to calculating statistics?

In [ ]:
income_tidy['prop_income'] = income_tidy.groupby('religion')['count'].transform(lambda x: x/x.sum()*100)

In [ ]:
income_tidy.head()

In [ ]:
income_tidy.loc[income_tidy['religion']=='Agnostic']

In [ ]:
income_tidy.loc[income_tidy['income']=='>150k'].sort_values('prop_income', ascending=False)

Now I can also easily add proportion of religions for each income group:

In [ ]:
income_tidy['prop_religion'] = income_tidy.groupby('income')['count'].transform(lambda x: x/x.sum()*100)

In [ ]:
income_tidy.loc[income_tidy['religion']=='Agnostic']

In [ ]:
income_tidy.loc[income_tidy['income']=='>150k'].sort_values('prop_religion',ascending=False)

Advantages of tidy data:

1. Adding new variables is easy and straightforward. It doesn't complicate the data structure or analysis.

2. When you have structurally missing data (like number of pregnancies for males), you can throw out some observations (in untidy table they will have to exist, although they make no sense)

In [ ]:
billboard_untidy = pd.read_csv(os.path.join('data','billboard.csv'))
billboard_untidy[billboard_untidy.columns[:10]].head()

In [ ]:
billboard_untidy.shape

If I wanted to look at, say, all songs which reached the 1st place, and see which song did it faster, there is no easy way of doing it in this data form. However, it is very easy to do with tidy data.

In [ ]:
billboard_tidy = pd.melt(billboard_untidy,id_vars=['year','artist',
                                                   'track','time','genre',
                                                   'date.entered','date.peaked'],
                        var_name='week',value_name='rank')

def keep_num(string):
    digits_list = [c for c in string if c.isdigit()]
    digits_string = ''.join(digits_list)
    number = int(digits_string)
    return number

billboard_tidy['week'].replace({s:keep_num(s) for s in billboard_tidy['week']}, inplace=True)
billboard_tidy.rename(columns={'date.entered':'entered', 
                               'date.peaked':'peaked'}, 
                      inplace=True)

billboard_tidy['entered'] = pd.to_datetime(billboard_tidy['entered'])
billboard_tidy['peaked'] = pd.to_datetime(billboard_tidy['peaked'])

billboard_tidy.head()

In [ ]:
billboard_tidy.shape

In [ ]:
billboard_tidy.loc[billboard_tidy['rank']==1].sort_values('week').head(10)

During tidying, each type of observational unit should be stored in its own table. This
is closely related to the idea of database normalization, where each fact is expressed in only
one place. If this is not done, it is possible for inconsistencies to occur.

The Billboard dataset described before actually contains observations on two types of
observational units: the song and its rank in each week. This manifests itself through the
duplication of facts about the song: `year`, `artist`, `track`, `time`, `genre`, `entered` and `peaked` are repeated for every `song` in each `week`.

But this examples is a bit complicated, so let's look at another one, which can be quite frequent in research. Afterwards we will get back to the billboard dataset to reorganize it as well.

In [ ]:
# generate subjects data

# number of subjects
n = 10

subjects_id = np.arange(n)

# if you have module `names`, it will generate names with it;
# otherwise, names will be just tokens 'Name Surname #'
try:
    import names
    subjects_names = []
    subjects_gender = []
    for g in ['male','female']:
        for i in np.arange(n/2):
            subjects_names.append(names.get_full_name(gender=g))
            subjects_gender.append(g)
            
except:
    subjects_names = ['Name Surname {}'.format(i+1) for i in range(n)]
    subjects_gender = ['male','female']*int(n/2)

subjects_age = np.random.randint(15, 85, size=n)

task_names = ['memory_matrix', 'labyrinth', 'math_game', 'names_memory', 'survey_score', 
              'session_1', 'session_2', 'session_3']

In [ ]:
# set subjects data into dataframe
df = pd.DataFrame({'id': subjects_id, 'name': subjects_names, 'gender': subjects_gender, 'age': subjects_age})

# generate scores for each task
for subj_id in df.index:
    for task in task_names:
        df.at[subj_id, task] = np.random.randint(100-df.at[subj_id, 'age'], 100)

In [ ]:
df

In [ ]:
df_tidy_1 = pd.melt(df, id_vars=['age','gender','name','id'],var_name='task',value_name='score')
df_tidy_1.sort_values('id')

In [ ]:
df_tidy_1.groupby('task')['score'].mean()

In [ ]:
df_tidy_1.groupby(['gender','task'])['score'].mean()

In [ ]:
df_tidy_1.groupby('age')['score'].mean()

In [ ]:
%matplotlib inline
df_tidy_1.groupby('age')['score'].mean().plot(marker='o',linestyle='')

For some applications this is fine. However, if I am interested in the sessions' scores, it is somewhat tedious to work with them. In a way, score on every task is actually a separate variable, because they are not directly comparable. Except for sessions' scores -- they are on the same task and they are directly comparable. How to organize this then? We could treat other scores, except for sessions, as separate variables, and keep scores for sessions and session # as another 2 variables, like so:

In [ ]:
df_tidy_2 = pd.melt(df, id_vars=['age','gender','name','id','memory_matrix', 
                     'labyrinth', 'math_game', 'names_memory', 'survey_score'],
                    var_name='session',value_name='score')

df_tidy_2['session'].replace({s:int(s[-1]) for s in df_tidy_2['session'].unique()}, inplace=True)
df_tidy_2.sort_values('id')

In [ ]:
df_tidy_2.groupby('math_game')['score'].mean()

In [ ]:
df_tidy_2.groupby('math_game')['score'].mean().plot(marker='o',linestyle='')

In [ ]:
df_tidy_2.groupby(['name','score'])['survey_score'].mean()

In [ ]:
df_tidy_2.groupby(['name'])['score'].mean()

In [ ]:
task_names[:-3]

In [ ]:
subj_df = df[['age','gender','name','id','memory_matrix', 'labyrinth', 
              'math_game', 'names_memory', 'survey_score']].set_index('id', drop='True')
subj_df

In [ ]:
scores_df = pd.melt(df[['id','session_1','session_2','session_3']],id_vars='id',var_name='session',value_name='score')
scores_df['session'].replace({s:int(s[-1]) for s in scores_df['session'].unique()}, inplace=True)
scores_df

In [ ]:
pd.merge(subj_df, scores_df, left_index=True, right_on='id')

Now, let's get back to the billboard dataset, and tidy it in the same manner.

In [ ]:
billboard_untidy.index.name = 'id'

songs_keys = billboard_untidy.keys()[:7]
songs_df = billboard_untidy[songs_keys]
songs_df.head()

In [ ]:
rank_keys = billboard_untidy.keys()[7:]
rank_df = billboard_untidy[rank_keys]
rank_df.index.name = 'id'
rank_df = rank_df.reset_index()
rank_df = pd.melt(rank_df, id_vars='id', var_name = 'week', value_name = 'rank')
rank_df['week'].replace({s:keep_num(s) for s in rank_df['week']}, inplace=True)
rank_df.head(20)

In [ ]:
rank_df.tail(20)

In [ ]:
pd.merge(songs_df, rank_df, left_index=True, right_on='id').head()

In [ ]:
pd.merge(songs_df, rank_df, left_index=True, right_on='id').shape

# Afterword

Tidy data is only worthwhile if it
makes analysis easier. **Tidy tools** take tidy datasets as input
and return tidy datasets as output. **Tidy tools** are useful because the output of one tool
can be used as the input to another. This allows you to simply and easily compose multiple
tools to solve a problem. Tidy data also ensures that variables are stored in a consistent,
explicit manner. This makes each tool simpler, because it does not need a Swiss Army knife
of parameters for dealing with different dataset structures.

Tools can be messy for two reasons: either they take messy datasets as input (**messy-input
tools**) or they produce messy datasets as output (**messy-output tools**). Messy-input tools are
typically more complicated than tidy-input tools because they need to include some parts of
the tidying process. This can be useful for common types of messy datasets, but it typically
makes the function more complex, harder to use and harder to maintain. Messy-output tools
are frustrating and slow down analysis because they cannot be easily composed and you must
constantly think about how to convert from one format to another.

We will see examples of messy tools and, more importantly, tidy tools in our future lessons.